# TP 4248 2021-2022 
## les membres du groupe
- fezeu Ghomsi Eugene Clotaire 17Q2864 
- Nono Mabou Wilfried Brondon 21S2817 brondonnono3@gmail.com
- NGBAYAFOU NGOUH Chéripha Cheriphalynette@gmail.com
- Junior Libii junior.libii@facsciences-uy1.cm

## binary naive bayes classification on sms dataset
on s'interesse ici a la presence ou l'abscende d'un mot dans corpus pour pour determiner si c'est un spam ou un ham

In [1]:
# load tools
# -*- coding: utf-8 -*-
!python -m ipykernel install --user --name=venv
!pip install nltk

Installed kernelspec venv in /home/eugene/.local/share/jupyter/kernels/venv


In [4]:
import os.path
from os import path
import getpass
filepath = ''
REPOPATH = 'fez2000/TP_4248.git'
FILENAME= 'spam.csv'
HOST='github.com'
if path.exists(f'./{FILENAME}'):
    filepath = f'./{FILENAME}'
elif path.exists(f'./datas/{FILENAME}'):
    filepath = f'./datas/{FILENAME}'
else:
    filepath = f'./datas/{FILENAME}'
    def load_from_git():
        def clone(): 
            GIT_PATH = f'https://{HOST}/{REPOPATH}' 
            !git clone {GIT_PATH} 'temp'
        def clone_by_token():
            GIT_TOKEN= getpass.getpass("votre token git") 
            GIT_PATH = f'https://{GIT_TOKEN}@{HOST}/{REPOPATH}' 
            !git clone {GIT_PATH} 'temp'
        def clone_by_username_and_password():
            GIT_USER_NAME= getpass.getpass("votre nom d'utilisateur git")
            GIT_PASSWORD= getpass.getpass("votre mot de pass git")
            GIT_PATH = f'https://{GIT_USER_NAME}:{GIT_PASSWORD}@{HOST}/{REPOPATH}'
            !git  clone  {GIT_PATH} 'temp' 
        if not path.exists(f'./temp/datas/{FILENAME}'):
            try:
                clone()
            except e:
                pass
        if not path.exists(f'./temp/datas/{FILENAME}'):
            try:
                clone_by_username_and_password()
            except e:
                pass
        if not path.exists(f'./temp/datas/{FILENAME}'):
            try:
                clone_by_token()
            except e:
                pass
        if not path.exists(f'./temp/datas/{FILENAME}'):     
            load_from_git()
        else:
            %cd './temp'

    load_from_git()


In [5]:

import pandas as pd
import re
import numpy as np
dataset = pd.read_csv(filepath, encoding="ISO-8859-1", usecols=['v1', 'v2'], dtype={'v1':str,'v2':str})
dataset.head()
dataset = dataset.dropna(how="any")
dataset = dataset.rename(columns={"v1": "target"})




In [6]:

import nltk
nltk.download('stopwords')
# ensemble de mots sans sens pas utilise pour la pertinence
# cette suppression offre un gain temps 
stopwords = nltk.corpus.stopwords.words('english')

#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()





[nltk_data] Downloading package stopwords to /home/eugene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def tonkenizer(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()).split()
def remove_stopwords(text):
    return[i for i in text if i not in stopwords]
# la lematisation reduit la taille du vecteur resultat et augmente la pertinence du model 
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def cleantext (text):
    return stemming(remove_stopwords(tonkenizer(text)))
def clean_and_binarise_text(row):
    return   [(row[0] == 'ham' and 1) or 0 , cleantext(row[1])]    

dataset_r = dataset.apply(clean_and_binarise_text, axis=1, result_type='broadcast')


/home/eugene/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [ ]:
wordset = [] 
def creer_wordset(words): 
    global wordset
    wordset = np.union1d(wordset, words)
dataset_r.v2.apply(creer_wordset);



In [ ]:
dataset = []
dataset_x = []
dataset_y = []
def vectorisation(l_doc):
    global wordset
    tf_diz = dict.fromkeys(wordset,0)
    for word in l_doc:
        tf_diz[word]=l_doc.count(word)
    return pd.Series(tf_diz.values(), index=tf_diz.keys()) 


dataset_x = pd.DataFrame(dataset_r.v2.apply(vectorisation))
#binarisation
dataset_x[dataset_x > 0] = 1
dataset_xy = dataset_x;
dataset_xy['target'] = dataset_r.target
dataset_xy.describe()
dataset_x=[]

In [ ]:
# split dataset
split_index= int(len(dataset_xy)*2/3.)
train_xy = dataset_xy.iloc[0:split_index]
test_xy = dataset_xy.iloc[split_index:]
dataset_r = []
targets = np.union1d(train_xy['target'],[])
(n,_) = train_xy.shape
target_probs = train_xy['target'].value_counts()/float(n)

In [ ]:
train_xy = train_xy.set_index(['target',train_xy.index])

In [ ]:
temp = dict.fromkeys(targets,{})
for target in targets:
    (n,_) = (train_xy.loc[[target]]).shape
    n = float(n)
    temp[target] = dict.fromkeys(train_xy.columns,0)
    for column in train_xy.columns:
        temp[target][column] = (train_xy.loc[[target]])[column] .value_counts()/ n;

In [ ]:
def predict (x): 
    output = dict.fromkeys(targets,0)
    for target in targets:
    i = 0
    prob = 1
    for column in train_xy.columns:
        try:
            prob *= temp[target][column][x[i]];
        except Exception as e:
            prob = 0
        break
        i = i + 1
        output[target] = prob * target_probs[target]
    return output

    
    

In [ ]:
from pandas.core.frame import DataFrame
def predict_classe(x):
    classe = predict(x)
    return max(classe, key=classe.get)
def confusion_matrix (test_dataset):
    output = DataFrame(np.zeros((targets.shape[0],targets.shape[0])), columns=targets,index=targets)  
    for i in test_dataset.index:
        row = test_dataset.loc[i] 
    output.loc[row.target][predict_classe(row.values)] += 1
    return output
    

In [ ]:

m = confusion_matrix(test_xy)



In [ ]:
ac = m.to_numpy().trace()/(m.sum().sum())
print(f'accurancy  est {ac}')
for target in targets:
    p = m.iloc[target][target]/m[target].sum();
    r = m.iloc[target][target]/m.iloc[target].sum();
    f = 2*r*p/(r+p)
    print(f'f mesure {target} est {f}')
    print(f'precision pour {target} est {p}')
    print(f'recall pour {target} est {p}')